# <div align="center"> 平台 API 测试</div>

In [2]:
from IPython.display import Markdown as md
import requests
import json
import consul
import time
import socket

In [3]:
consul_addr = !echo $(curl -s http://txt.go.sohu.com/ip/soip)| grep -P -o -i "(\d+\.\d+.\d+.\d+)"
consul_port = 8500
consul_addr = consul_addr[0]

In [4]:
# 获取本机器IP
def get_host_ip():
    ip=''
    try:
        s = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
        s.connect(('8.8.8.8',80))
        ip = s.getsockname()[0]
        s.close()
    except Exception as err:
        print(err)
    return ip

# API异步接口将过程中产出的数据同步到consul数据中心, 调用该接口可从中心获取数据 (数据中心仅供开发使用)
def query_async_data(key, wait = 1):
    time.sleep(wait)
    try:
        client = consul.Consul(consul_addr, consul_port)
        _, data = client.kv.get(key)
        return str(data['Value'], encoding="utf-8")
    except:
        return 'Query nothing, try again later!'
    
# 格式化打印json字符串
def print_json(text):
    if isinstance(text, str):
        print(json.dumps(json.loads(text), indent=4))
    else:
        print(json.dumps(text, indent=4))

In [5]:
host = get_host_ip()
port = 8119

user = 'test'
uuid = '123456'

## 1. 平台状态`/k12ai/platform/stats`


A. 输入

```json
{
    "username": "test",  // [M] 登陆平台用户名 (未使用)
    "password": "test",  // [M] 登陆平台用户密码 (未使用)
    "async": true,  // [O] 默认同步, 即值为false, api调用过程方式
    "query": {  //  [O] 默认query所有项
        "cpus": true, // [O] k12ai主机cpu信息 
        "gpus": true, // [O] k12ai主机gpu信息
        "disks": false, // [O] k12ai主机磁盘信息
        "containers": true // [O] k12ai主机上正在运行的label为"k12ai.service.name"的container信息
    }
}
```

B. 输出

```json
{
    "version": "0.1.0", // [M] 返回json格式的版本号, 兼容版本时使用
    "type: "info",
    "tag": "platform", // [M] 标识输出结果的服务TAG, 可以用来后续消息过滤 
    "op": "stats", // [M] 标识输出结果由哪个功能产出, 一般统一为"/k12ai/{{tag}}/{{op}}"的形式
    "timestamp": , // [M]
    "datetime": , // [M]
    "info": { // [M] 输出结果的正文内容
        "cpu_percent": 47.4,  // [C] k12ai主机cpu使用率, 不区分多cpu 
        "cpu_percent_list": [ // [C] k12ai主机cpu使用率, 每个cpu的使用率
            63.6,
            31.1
        ],
        "cpu_memory_total":, // [C] k12ai主机memory总大小(Byte)
        "cpu_memory_usage":, // [C] k12ai主机memory已使用(Byte)
        "cpu_memory_percent": , // [C] k12ai主机memory使用率
        "gpus": [  // [C] k12ai主机gpu信息, 每个gpu信息列表
            {
                "name": "", // [C] gpu名字, 厂商
                "gpu_percent": , // [C] gpu使用率
                "gpu_memory_total": , // [C] gpu显存总大小(Byte)
                "gpu_memory_usage": , // [C] gpu显存已使用(Byte)
                "gpu_memory_percent": // [C] gpu显存使用率
            }
        ],
        "containers": [ // k12ai主机正在运行的关于训练相关的container信息
            {
                "id": "", // container的ID
                "cpu_percent": , // container的cpu使用率
                "cpu_memory_total":,  // container的memory总大小
                "cpu_memory_usage":,  // container的memory已使用
                "cpu_memory_percent":, // container的memory利用率
                "op": "",  // container正在执行的操作,如:"train.start", "predict.start"等
                "user": "", // container用户ID
                "service_uuid": "" // container执行的任务/服务ID.
            }
        ]
    }
}
```

In [8]:
data = '''{
    "username": "test",
    "password": "test",
    "async": false,
    "query": {
        "cpus": true,
        "gpus": true,
        "disks": false,
        "containers": true
    }
}'''

api = 'http://%s:%d/k12ai/platform/stats' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success",
    "message": {
        "cpu_percent": 6.6,
        "cpu_percent_list": [
            6.2,
            5.7,
            6.2,
            6.7,
            6.6,
            7.4,
            6.8,
            7.5
        ],
        "cpu_memory_total": 50648887296,
        "cpu_memory_usage": 3674685440,
        "cpu_memory_percent": 8.7,
        "gpus": [
            {
                "name": "Tesla P40",
                "gpu_percent": 92.0,
                "gpu_memory_total": 22919.0,
                "gpu_memory_usage": 4443.0,
                "gpu_memory_percent": 19.39
            }
        ],
        "containers": [
            {
                "id": "99558f81bc3b",
                "cpu_percent": 180.25,
                "cpu_memory_total": 50648887296,
                "cpu_memory_usage": 2470383616,
                "cpu_memory_percent": 4.88,
                "op": "train",
                "user": "test",
                "serv

In [7]:
md("[异步操作Consul平台结果](http://{}:8500/ui/k12ai/kv/platform/admin/stats/info/edit)".format(consul_addr))

[异步操作Consul平台结果](http://116.85.5.40:8500/ui/k12ai/kv/platform/admin/stats/info/edit)

```json
{
    "version": "0.1.0",
    "type": "info",
    "tag": "platform",
    "op": "stats",
    "timestamp": 1574420133259,
    "datetime": "2019-11-22 18:55:33",
    "info": {
        "cpu_percent": 6.3,
        "cpu_percent_list": [
            5.5,
            7.0
        ],
        "cpu_memory_total": 4143292416,
        "cpu_memory_usage": 1265950720,
        "cpu_memory_percent": 36.7,
        "gpus": [
            {
                "name": "Tesla P4",
                "gpu_percent": 0.0,
                "gpu_memory_total": 7611.0,
                "gpu_memory_usage": 10.0,
                "gpu_memory_percent": 0.13
            }
        ],
        "containers": [
            {
                "id": "30a879f11ec4",
                "cpu_percent": 105.44,
                "cpu_memory_total": 4143292416,
                "cpu_memory_usage": 355155968,
                "cpu_memory_percent": 8.57,
                "op": "train",
                "user": "test",
                "service_uuid": "123456"
            }
        ]
    }
}
```

In [7]:
print(query_async_data('platform/admin/stats/info'))

{
    "version": "0.1.0",
    "type": "info",
    "tag": "platform",
    "op": "stats",
    "timestamp": 1575345355854,
    "datetime": "2019-12-03 11:55:55",
    "info": {
        "cpu_percent": 0.7,
        "cpu_percent_list": [
            0.6,
            0.6,
            0.8,
            0.9,
            0.6,
            0.7,
            0.7,
            0.8
        ],
        "cpu_memory_total": 50648887296,
        "cpu_memory_usage": 1228599296,
        "cpu_memory_percent": 3.6,
        "gpus": [
            {
                "name": "Tesla P40",
                "gpu_percent": 0.0,
                "gpu_memory_total": 22919.0,
                "gpu_memory_usage": 0.0,
                "gpu_memory_percent": 0.0
            }
        ],
        "containers": []
    }
}


## 2. 平台控制/k12ai/platform/control

A. 输入

```json
{
    "username": "", // [M] 登陆k12ai service的User
    "password": "", // [M] 登陆k12ai service的Code
    "op": "",  // [M] 控制命令, 如, 停止某个container任务"container.stop"
    "async": true, // [M] 控制命令的是否异步操作, 提醒: 有些命令操作是耗时的, 需要采用异步方式
    "params": { // [C] 命令的参数, 根据命令不同, 内部结构也会不同
    }
}
```

B. 输出

```json
{
    "code": 100200,
    "descr": "task service success",
    "message": "container id f3726eae19d6 stop success"
}
```

### 2.1 停止container

In [8]:
try:
    result = json.loads(query_async_data('platform/admin/stats/info'))
    cid = result['info']['containers'][0]['id']
except:
    cid = 'testid'
    
cmd = 'container.stop'

data = '''{
    "username": "test",
    "password": "test",
    "op": "%s",
    "async": false,
    "params": {
        "id": "%s"
    }
}''' % (cmd, cid)

api = 'http://%s:%d/k12ai/platform/control' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100202,
    "descr": "task service start fails",
    "message": "container id testid is not found"
}


In [9]:
print(query_async_data('platform/admin/control/container.stop'))

{
    "version": "0.1.0",
    "type": "container.stop",
    "tag": "platform",
    "op": "control",
    "timestamp": 1575345357971,
    "datetime": "2019-12-03 11:55:57",
    "container.stop": "container id testid is not found"
}
